<a href="https://colab.research.google.com/github/leon123858/YOLO_exercise/blob/master/Mask_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#引入一個開源運算框架 
! git clone https://github.com/pjreddie/darknet.git
# 修改make file
! head darknet/Makefile
!sed -i "s/GPU=0/GPU=1/g" darknet/Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" darknet/Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" darknet/Makefile
#編譯
! cd darknet; make

In [ ]:
#抓取訓練資料
!wget https://tinyurl.com/y6d6p24o

In [6]:
#更新成可用的檔名
!mv /content/y6d6p24o /content/medical-masksdataset.zip

mv: cannot stat '/content/y6d6p24o': No such file or directory


In [8]:
#解壓縮
! unzip /content/medical-masksdataset.zip > data_unzip.log

In [ ]:
#查看圖片對應的XML 用支援YOLO的標註器完成
! cat /content/labels/012106.jpg_1140x855.xml

In [11]:
#建立資料夾
! rm -rf /content/yolo
! mkdir /content/yolo

In [32]:
#建置給YOLO format的函數-> 把data 的 label 給yolo用
from bs4 import BeautifulSoup
import os
import shutil

status_dic = {'good':2,'bad':1,'none':0}
def getYoloFormat(filename,label_path,img_path,yolo_path,newname):
  with open(label_path+ filename, 'r') as f:
    # read xml
    soup = BeautifulSoup(f.read(),'xml')
    imgname = soup.select_one('filename').text
    image_w = int(soup.select_one('width').text)
    image_h = int(soup.select_one('height').text)
    ary = []
    for obj in soup.select('object'):
      #轉換標籤位置
      xmin = int(obj.select_one('xmin').text)
      xmax = int(obj.select_one('xmax').text)
      ymin = int(obj.select_one('ymin').text)
      ymax = int(obj.select_one('ymax').text)
      objclass = status_dic.get(obj.select_one('name').text)
      #照YOLO的格式訂好x,y,w,h
      x = (xmin + (xmax-xmin)/2) * 1.0 / image_w
      y = (ymin + (ymax-ymin)/2) * 1.0 / image_h
      w= (xmax-xmin) * 1.0 / image_w
      h = (ymax-ymin) * 1.0 / image_h
      ary.append(' '.join([str(objclass), str(x), str(y), str(w), str(h)]))
    #圖片本來在image裡面，把圖片移到yolo資料夾下
    #同時把yolo參數檔寫到yolo之下
    if os.path.exists(img_path + imgname):
        shutil.copyfile(img_path + imgname, yolo_path + newname + '.jpg')
        with open(yolo_path + newname + '.txt', 'w') as f:
          f.write('\n'.join(ary))



In [33]:
#使用函數 把訓練圖片和標籤(處理後變txt)放入同一個目錄下
import os
labelpath = '/content/labels/'
imgpath = '/content/images/'
yolopath = '/content/yolo/'
ary = []
for idx,f in enumerate(os.listdir(labelpath)):
  try:
    getYoloFormat(f,labelpath,imgpath,yolopath,str(idx))
  except Exception as e:
    print(e)

float division by zero


In [34]:
#看一下標籤轉換後的樣子
! cat /content/yolo/0.txt

2 0.20944444444444443 0.535 0.07444444444444444 0.09285714285714286
2 0.5677777777777778 0.13 0.09555555555555556 0.11142857142857143

In [35]:
#建立資料夾
! rm -rf /content/cfg_mask
! mkdir /content/cfg_mask

In [36]:
#計算資料總數 且8:2
import os
datasets = ['/content/yolo/' + f for f in os.listdir('/content/yolo/') if not f.endswith('.txt')]
len(datasets)*0.8

541.6

In [38]:
#資料分兩邊擺
with open('/content/cfg_mask/train.txt','w') as f:
  f.write('\n'.join(datasets[0:540]))
with open('/content/cfg_mask/test.txt','w') as f:
  f.write('\n'.join(datasets[540:]))

In [ ]:
#取得設定檔
!wget https://drive.google.com/u/1/uc?id=1x1Hz131ZZsQKw55vM8CeOp_L-hRVlcbn&export=download
!wget https://drive.google.com/u/1/uc?id=1yuK3_SuS5YWZpV7fVWyVK6c_Au43nhus&export=download

In [40]:
#用旁邊改名 再移動位置
! mv '/content/obj.data' '/content/cfg_mask/obj.data'
! mv '/content/obj.names' '/content/cfg_mask/obj.names'

In [41]:
#看一下設定檔
! head /content/cfg_mask/obj.names

none
bad
good


In [44]:
# 建立權重資料夾
! mkdir /content/cfg_mask/weights

In [45]:
#修改參數
#先複製原來的檔案到 cfg_mask
! cp /content/darknet/cfg/yolov3-tiny.cfg /content/cfg_mask/

In [48]:
#看一下設定檔
! sed -n -e 127p -e 135p -e 171p -e 177p /content/cfg_mask/yolov3-tiny.cfg

filters=24
classes=3
filters=24
classes=3


In [47]:
#我們的class 只有三類 戴 沒戴 戴的不正常
! sed -i '127s/255/24/' /content/cfg_mask/yolov3-tiny.cfg
! sed -i '135s/80/3/' /content/cfg_mask/yolov3-tiny.cfg
! sed -i '171s/255/24/' /content/cfg_mask/yolov3-tiny.cfg
! sed -i '177s/80/3/' /content/cfg_mask/yolov3-tiny.cfg

In [50]:
# 抓取模型 (我們的模型構築在已經訓練好的一個模型上) 再複製到資料夾
#!wget https://pjreddie.com/media/files/darknet53.conv.74
! cp "/content/darknet53.conv.74" /content/cfg_mask/

In [52]:
! darknet/darknet detector train /content/cfg_mask/obj.data /content/cfg_mask/yolov3-tiny.cfg /content/cfg_mask/darknet53.conv.74

CUDA Error: no CUDA-capable device is detected
darknet: ./src/cuda.c:36: check_error: Assertion `0' failed.
